In [104]:
import re

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Elterngeld: Basisdaten von destatis

Hier führen wir einige Grunddaten ein, die bei destatis verfügbar sind.

## Geburten

Die Zahl der Geburten dient als Grundlage, ohne die Schwankungen beim Elterngeldbezug nicht zu verstehen sind.
Da die Spanne zwischen Bremen und NRW sehr groß ist und wir uns mehr für Unterschiede in den relativen Geburtenraten zwischen Ländern und über die Jahre hinweg interessieren, skalieren wir die Geburtenrate auf "Geburten je 100.000 Einwohner:innen".
Die Tabelle enthält auch das Geschlecht des Kindes, das ignorieren wir hier.

In [105]:
gb = pd.read_parquet("../data/processed/geburten.parquet")
gb = gb.groupby(["jahr", "land"]).geburten.sum().to_frame().reset_index()

ewz = pd.read_parquet("../data/processed/ewz.parquet")

gb = pd.merge(gb, ewz, on=["jahr", "land"])
gb["geburten_pro_1000"] = gb.geburten / gb.ewz * 1000
# gb.land = gb.land.str.replace("-", "-<br>")

fig = go.Figure()

for land, grp in gb.groupby("land"):
    fig.add_trace(
        go.Scatter(
            x=grp.jahr,
            y=grp.geburten_pro_1000,
            mode="markers+lines",
            name=land
        ),
    )

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(t=20, r=20, b=20, l=20)
)

fig.show()

Interessant ist wie 2021/22 in den alten Bundesländern die Zahl wieder steigt, aber in den neuen sich der Abwärtstrend nicht umkehrt.
Möglicherweise gibt es Geschlechterunterschiede in Wanderungsbewegungen?
Die Geburtenrate pro Mutter wäre interessant.

## Empfangende von Elterngeld


In [125]:
eg = pd.read_parquet("../data/processed/eg_empf.parquet")

eg = eg.loc[eg.art.eq("Insgesamt")]
eg = eg.loc[eg.fm.ne("Insgesamt")]

eg = eg.groupby(["jahr", "land", "fm", "art"]).sum().drop("quartal", axis=1).reset_index()

eg["grp_display"] = eg.apply(
    lambda row: f"{'Mütter' if row.fm=='weiblich' else 'Väter'} in {row.land}",
    axis=1
)

eg = eg.sort_values(by=["jahr", "land", "fm"], ascending=[True, True, False]).reset_index(drop=True)
land_clr = {
    'Schleswig-Holstein': '#1f77b4',
    'Hamburg': '#ff7f0e',
    'Niedersachsen': '#2ca02c',
    'Bremen': '#d62728',
    'Nordrhein-Westfalen': '#9467bd',
    'Hessen': '#8c564b',
    'Rheinland-Pfalz': '#e377c2',
    'Baden-Württemberg': '#7f7f7f',
    'Bayern': '#bcbd22',
    'Saarland': '#17becf',
    'Berlin': '#aec7e8',
    'Brandenburg': '#ffbb78',
    'Mecklenburg-Vorpommern': '#98df8a',
    'Sachsen': '#ff9896',
    'Sachsen-Anhalt': '#c5b0d5',
    'Thüringen': '#c49c94',
}

fm_line = {
    "weiblich": "solid",
    "männlich": "dash",
}

fig = go.Figure()

for var, grp in eg.groupby(["land", "fm"]):
    fig.add_trace(
        go.Scatter(
            x=grp.jahr,
            y=grp.pers,
            mode="markers+lines",
            line=dict(
                dash=fm_line[var[1]],
                color=land_clr[var[0]]
            ),
            name=var[0],
            showlegend=var[1]=="weiblich",
            legendgroup=var[0],
            visible=True if var[0] == "Berlin" else "legendonly",
            customdata=grp.grp_display.values,
            hovertemplate="%{customdata}: %{y:f}<extra></extra>"
        )
    )

fig.update_layout(
    hovermode="x unified",
    width=1000,
    height=550,
    margin=dict(t=50, r=20, b=20, l=20),
    yaxis=dict(
        range=[0, eg.pers.max()],
        tickformat=".0f"
    ),
    title="Empfänger:innen von Elterngeld"
)

fig.show()

# Empfänger:innen von Elterngeld in Prozent Anteil an Geburten

In [118]:
egb = pd.merge(
    eg,
    gb[["jahr", "land", "geburten"]],
    on=["jahr", "land"],
    how="left"
)
egb["eg_rate"] = egb.pers / egb.geburten * 100

fig = go.Figure()

for var, grp in egb.groupby(["land", "fm"]):
    fig.add_trace(
        go.Scatter(
            x=grp.jahr,
            y=grp.eg_rate,
            mode="markers+lines",
            line=dict(
                dash=fm_line[var[1]],
                color=land_clr[var[0]]
            ),
            name=var[0],
            showlegend=var[1]=="weiblich",
            legendgroup=var[0],
            visible=True if var[0] == "Berlin" else "legendonly",
            customdata=grp.grp_display.values,
            hovertemplate="%{customdata}: bei %{y:.1f}% der geborenen Kinder<extra></extra>"
        )
    )

fig.update_layout(
    hovermode="x unified",
    width=1000,
    height=800,
    margin=dict(t=20, r=20, b=20, l=20),
    yaxis=dict(range=[0, egb.eg_rate.max()])
)

fig.show()